In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.layers import *
import numpy as np

In [146]:
from google.colab import drive

drive.mount('/content/drive')
RunningInCOLAB = 'google.colab' in str(get_ipython())

path = "./data_collection/snake2_output.csv"
if RunningInCOLAB: path = "/content/drive/MyDrive/24Winter/cs320-w24/cs320-final-project/data_collection/snake2_output.csv"
df_import = pd.read_csv(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [147]:
# GLOBAL VARIABLES
USE_DUMMY_FEATURES = False

In [148]:
df_import = pd.read_csv(path)
df_import.head()
df = df_import
df.head()

,obstacleL,obstableF,obstacleR,direction_choice,game_over
0,0,0,0,2,0
1,0,0,0,2,0
2,0,0,0,1,0
3,0,0,0,1,0
4,0,0,0,0,0


In [149]:
# selecting better training set:
df_game_over = df[df['game_over'] == 1]
df_game_not_over = df[df['game_over'] == 0]
df_game_not_over_sample = df_game_not_over.sample(n=500, random_state=42)
df_game_not_over_sample

# this df will have more equal amounts of dead/alive examples
df_sample = pd.concat([df_game_over, df_game_not_over_sample], ignore_index=True)


In [150]:
dummy_columns = None

df = df_sample

if USE_DUMMY_FEATURES:
  dummy_columns = pd.get_dummies(df['direction_choice'], prefix='direction_choice')

def dir_num_to_array(direction: int):
  t = [0,0,0]
  t[direction] = 1
  return t

In [151]:
df['direction_choice'] = df['direction_choice'] - 1
df

,obstacleL,obstableF,obstacleR,direction_choice,game_over
0,1,0,0,-1,1
1,0,1,0,0,1
2,0,1,0,0,1
3,1,0,0,-1,1
4,1,0,0,-1,1
...,...,...,...,...,...
755,0,0,0,1,0
756,0,0,1,-1,0
757,0,0,0,1,0
758,0,0,0,-1,0


In [152]:
dummy_columns

,direction_choice_0,direction_choice_1,direction_choice_2
0,1,0,0
1,0,1,0
2,0,1,0
3,1,0,0
4,1,0,0
...,...,...,...
755,0,0,1
756,1,0,0
757,0,0,1
758,1,0,0


In [153]:
if USE_DUMMY_FEATURES:
  df = pd.concat([df, dummy_columns], axis=1)
  df.drop('direction_choice', inplace=True, axis=1)

In [154]:
df.head()

,obstacleL,obstableF,obstacleR,game_over,direction_choice_0,direction_choice_1,direction_choice_2
0,1,0,0,1,1,0,0
1,0,1,0,1,0,1,0
2,0,1,0,1,0,1,0
3,1,0,0,1,1,0,0
4,1,0,0,1,1,0,0


In [107]:
X = df.drop('game_over', axis=1).to_numpy()
y = df['game_over'].to_numpy()
print(X.shape, "\n", y.shape)

X_feature_shape = X.shape[1]
X_feature_shape

(760, 4) 
 (760,)


4

In [108]:
# create testing and training data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [158]:
# init model


model = keras.Sequential()

# input layer
model.add(Input(shape=(X_feature_shape,)))
model.add(Dense(units=(5), activation='sigmoid'))
# model.add(Dense(units=(5)))
model.add(Dense(units=(1), activation='sigmoid'))

optimizer = keras.optimizers.Adam(learning_rate=1e-1)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 5)                 25        
                                                                 
 dense_24 (Dense)            (None, 1)                 6         
                                                                 
Total params: 31 (124.00 Byte)
Trainable params: 31 (124.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [159]:
model.fit(X_train, y_train, epochs=40, batch_size=32)

Epoch 1/40
17/17 [==============================] - 1s 5ms/step - loss: 0.5494 - accuracy: 0.7124
Epoch 2/40
17/17 [==============================] - 0s 4ms/step - loss: 0.3162 - accuracy: 0.8929
Epoch 3/40
17/17 [==============================] - 0s 4ms/step - loss: 0.2211 - accuracy: 0.9229
Epoch 4/40
17/17 [==============================] - 0s 4ms/step - loss: 0.1886 - accuracy: 0.9455
Epoch 5/40
17/17 [==============================] - 0s 4ms/step - loss: 0.1654 - accuracy: 0.9380
Epoch 6/40
17/17 [==============================] - 0s 4ms/step - loss: 0.1345 - accuracy: 0.9455
Epoch 7/40
17/17 [==============================] - 0s 5ms/step - loss: 0.1046 - accuracy: 0.9568
Epoch 8/40
17/17 [==============================] - 0s 5ms/step - loss: 0.0753 - accuracy: 0.9737
Epoch 9/40
17/17 [==============================] - 0s 4ms/step - loss: 0.0557 - accuracy: 0.9831
Epoch 10/40
17/17 [==============================] - 0s 5ms/step - loss: 0.0421 - accuracy: 1.0000
Epoch 11/40
17/17 [

In [189]:
model.save("/content/drive/MyDrive/24Winter/cs320-w24/cs320-final-project/models/snake2_possiblemvp_functional1.keras")

In [161]:
model.evaluate(X_train, y_train)

17/17 [==============================] - 0s 2ms/step - loss: 0.0023 - accuracy: 1.0000


[0.002265308052301407, 1.0]

In [162]:
y_preds = model.predict(X_test)

y_preds[y_preds > 0.5] = 1
y_preds[y_preds <= 0.5] = 0

8/8 [==============================] - 0s 3ms/step


In [163]:
# overall accuracy

y_test_reshape = y_test.reshape((y_test.shape[0], 1))

print("Overall Accuracy:")
len(y_preds[np.where(y_preds == y_test_reshape)])/len(y_preds)

Overall Accuracy:


1.0

In [164]:
# accuracy where game_over == 1
y_preds_gameover_subset = y_preds[X_test[:,3] == 1]
y_test_subset = y_test[X_test[:,3] == 1]
y_test_subset = y_test_subset.reshape((y_test_subset.shape[0], 1))

len(y_preds_gameover_subset[y_preds_gameover_subset == y_test_subset])/len(y_test_subset)

1.0

In [182]:
y_test

array([0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0])

In [181]:
X_test[2]

array([0, 1, 0, 0])

In [167]:
model.predict(X_test[1].reshape((1,4)))

1/1 [==============================] - 0s 16ms/step


array([[0.00084456]], dtype=float32)

In [188]:
obs = [1,0,0]
direction = -1
input = np.concatenate([obs, [direction]]).reshape((1,4))

print(input)


model.predict(input)

[[ 1  0  0 -1]]
1/1 [==============================] - 0s 16ms/step


array([[0.9960614]], dtype=float32)

In [177]:
len(y_preds[y_preds > 0])

81

In [178]:
len(y_test[y_test > 0])

81

In [171]:
np.where(y_p)

NameError: name 'y_p' is not defined

In [172]:
df_died = df[df['game_over'] == 1]
df_died_L = df_died[df_died['obstacleL'] == 1]
df_died_F = df_died[df_died['obstableF'] == 1]
df_died_R = df_died[df_died['obstacleR'] == 1]

In [173]:
len_full = len(df_died_F)
np.array([
    len(df_died_L)/len_full,
    len(df_died_F)/len_full,
    len(df_died_R)/len_full,
])

array([0.41721854, 1.        , 0.36423841])